TownCharts.com will provide the initial set of variables.
Other variables might be merged after ...

First collect relevant variables from each figures that display consistent information from the housing tab 

And then automate since its a structured website for cities in california.

In [21]:
import requests
import urllib
import re
from bs4 import BeautifulSoup
from googlesearch import search
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
import time
import pickle

In [5]:
cities_data = pd.read_csv('C:\\Users\\Crist\\Towncharts_Project\\California_Housing_Project\\datasets\\cal_cities_lat_long.csv')

In [6]:
cities_data.Name.size

458

In [7]:
city_list = cities_data.Name.tolist()
list_of_cities = []
for city in city_list:
    new = city.replace(' ','-')
    list_of_cities.append(new)

In [8]:
list_of_cities

['Adelanto',
 'Agoura-Hills',
 'Alameda',
 'Albany',
 'Alhambra',
 'Aliso-Viejo',
 'Alturas',
 'Amador-City',
 'American-Canyon',
 'Anaheim',
 'Anderson',
 'Angels',
 'Antioch',
 'Arcadia',
 'Arcata',
 'Arroyo-Grande',
 'Artesia',
 'Arvin',
 'Atascadero',
 'Atwater',
 'Auburn',
 'Avalon',
 'Avenal',
 'Azusa',
 'Bakersfield',
 'Baldwin-Park',
 'Banning',
 'Barstow',
 'Beaumont',
 'Bell-Gardens',
 'Bellflower',
 'Belmont',
 'Belvedere',
 'Berkeley',
 'Beverly-Hills',
 'Big-Bear-Lake',
 'Biggs',
 'Bishop',
 'Blue-Lake',
 'Blythe',
 'Bradbury',
 'Brawley',
 'Brea',
 'Brentwood',
 'Brisbane',
 'Buellton',
 'Buena-Park',
 'Burbank',
 'Burlingame',
 'Calabasas',
 'Calexico',
 'California-City',
 'Calimesa',
 'Calipatria',
 'Calistoga',
 'Camarillo',
 'Campbell',
 'Canyon-Lake',
 'Capitola',
 'Carlsbad',
 'Carmel-By-the-Sea',
 'Carpinteria',
 'Carson',
 'Cathedral-City',
 'Ceres',
 'Cerritos',
 'Chico',
 'Chino',
 'Chino-Hills',
 'Chowchilla',
 'Chula-Vista',
 'Citrus-Heights',
 'Claremont',
 

In [9]:
clean_links = []
missing_cities = []

In [ ]:
for city in list_of_cities:
    #only get links for cities not in clean_links
        backup_url = "https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html".format(city)
        query  = "'https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html'".format(city)
        query = urllib.parse.quote_plus(query) # Format into URL encoding
        number_result = 3
        
        ua = UserAgent()
        
        google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number_result)
        t0 = time.time()
        response = requests.get(google_url, {"User-Agent": ua.random})
        response_delay = time.time() - t0
        time.sleep(10*response_delay)
        soup = BeautifulSoup(response.text, "html.parser")
        
        result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})
        
        links = []
        for r in result_div:
            
            try:
                link = r.find('a', href = True)

                if link != '':
                    links.append(link['href'])
            except:
                continue
        flag = True
        for i, l in enumerate(links):
            clean = re.search('\/url\?q\=(.*)\&sa',l)
            if clean is None:
                continue
            elif city in clean.group(1):
                clean_links.append(clean.group(1))
                flag = False
        if flag:
            missing_cities.append(city)
            clean_links.append(backup_url)

Not perfect so we will:
    *remove identical strings
    *remove strings that have the word county in them
    *track http error 404 if our self made url fails

Pickle the output

In [ ]:
# outfile = open('my_clean_links','wb')
# pickle.dump(clean_links,outfile)
# outfile.close()

# outfile = open('unfound_urls','wb')
# pickle.dump(missing_cities,outfile)
# outfile.close()

Load the the clean_links

In [8]:

# infile = open('my_clean_links','rb')
# clean_links = pickle.load(infile)
# infile.close()

# infile = open('unfound_urls','rb')
# unfound_urls = pickle.load(infile)
# infile.close()

In [9]:
clean_links = pd.Series(clean_links)

In [10]:
# #remove county results
# without_counties = pd.Series(clean_links)
# without_counties = without_counties.mask(pd.Series(clean_links).str.contains('County'))
# without_counties = without_counties.dropna()

In [11]:
#delete duplicates
city_urls = clean_links.drop_duplicates()

In [12]:
#need to make sure to reset the indexes
city_urls = city_urls.reset_index(drop=True)

Housing tab integrity check

In [13]:
indexes_to_replace = []

In [14]:

for i,v in enumerate(city_urls):
    v = pd.Series(v)
    check = v.str.extract('\/California\/(.*?)\/',expand=False)
    if 'Housing' not in check[0]:
         print(check[0])
         indexes_to_replace.append(i)


Economy
Economy
Demographics
Economy


In [15]:
indexes_to_replace

[22, 43, 74, 77]

Consistent format with city not CDP or CDD

In [16]:
for i,v in enumerate(city_urls):
    v=pd.Series(v)
    check = re.findall('City',v[0])
    check2 = re.findall('city',v[0])
    if ( (check == []) & (check2 == [])):
        indexes_to_replace.append(i)

In [17]:
indexes_to_replace

[22, 43, 74, 77, 3, 29, 40, 51, 67, 73, 85, 90]

**scratch work**

In [18]:
# unintended_urls = pd.Series()
# for i in indexes_to_replace:
#     unintended_urls = unintended_urls.append( pd.Series([city_urls[i]]),ignore_index=True )
# for i in unintended_urls:
#     print(i)
# len(unintended_urls)

In [19]:
# appropriate_names = []
# for city in list_of_cities:
#     if not unintended_urls[unintended_urls.str.contains(city)].empty:
#         appropriate_names.append(city)
# appropriate_names

In [20]:
# fix_cities = []
# mycount = 0
# while mycount != len(unintended_urls):
#     for i in unintended_urls:
#         for l in appropriate_names:
#             if l in i:
#                 fix_cities.append(l)
                
#                 mycount +=1
# fix_cities

In [21]:
# to_fix = pd.Series(fix_cities,index = indexes_to_replace)
# to_fix

In [22]:
# for i in indexes_to_replace:
#     v = pd.Series(city_urls[i])
#     check = v.str.extract(r'nia\/.*?\/(.*?)-')
#     print(check)
# take the indexes out add backup url only 
# for i in list:
#     city_urls.str.contains('name')

**scratch work done**

Going to find which cities google was not able to find a suitable url.
In effect gathering the city names and indexes so we may use the backup url in place of the current value
(this could be implemented in the gathering of results as well)


In [23]:
#once we take them out 
incomplete_urls = city_urls.drop(labels = indexes_to_replace)

In [24]:
cities_to_add = []
for city in list_of_cities:
    if incomplete_urls[incomplete_urls.str.contains(city)].empty:
        cities_to_add.append(city)
cities_to_add

['Avalon', 'Chowchilla', 'Coachella']

In [25]:
fill_indexes =[]
for city in cities_to_add:
    fill_indexes.append(city_urls[city_urls.str.contains(city)].index[0])
fill_indexes

[22, 77, 85]

In [26]:
city_urls = city_urls.drop(labels = indexes_to_replace)

Actually take them out add them where they were and then reset index
<br>
can add them where they were before (22,77,85) with our backup url
anywhere but since its just three we can

Note:city_urls indexes have not been reset


In [27]:
cities_to_fix = pd.Series(cities_to_add,index = fill_indexes)
cities_to_fix

22        Avalon
77    Chowchilla
85     Coachella
dtype: object

In [28]:
for i,v in cities_to_fix.items():
    city_urls = city_urls.append( pd.Series(["https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html".format(v)],index = [i]) )

In [29]:
city_urls.reset_index(inplace= True,drop=True)
len(city_urls)

458

In [197]:
#double check
# #delete duplicates if accidentally created any
# searchurls = cop_city_urls.drop_duplicates()
# len(searchurls)

458

Now we actually scrape the desired attributes:
<br>
First up is the housing the variables. Out of the 44 figures 18(3,9,13,16,19,20,22,23,25,26,27,31,32,33,34,35,38,39) will not be included due to incosistencies. Some cities might not have the figure information and an NA will be put in place. 

Note figure 24:will have several variables

Data dictionary: provided by Towncharts.com

* city_name: Name of city
* housing_units:Total number of housing units in the area
* housing_density : The number of housing units per square mile in the area
* change_hunits: Change in housing units from 2010 to 2017
* percent_of_rent_to_total: The percent of all occupied housing units that are rental housing units (%)
* owned_homes: The percent of all occupied housing units that are owned housing units (%)
* med_homeval: Median home value i.e how much property is worth( house and lot, mobile homes and lot, or condominium unit) if it was for sale \\$

* med_rental_rate: the median monthly rental amount for a rental unit in this area \\$

* med_owner_cost: The monthly cost of housing for property owners including mortgage payment, taxes, insurance,and utilities.\\$

* med_own_cost_aspercentof_income: The monthly owner cost as a percent of the household income. This measure is an excellent way to understand how affordable housing is for owners in an area (%)
* med_hval_aspercentof_medearn: How much the property is worth(house and lot, mobile home and lot, or condominium unit) if it was for sale as a percent of the median earnings for a worker in the area (%)
* med_hcost_ownmortg: Median housing cost for homeowners with a mortgage(including the cost of the mortgage or other debt) \\$
* med_hcost_own_wo_mortg: Median housing cost for homeowners who do not have a mortgage. This isolates the cost of ownership seperate from the financing cost of debt \\$
* hcost_aspercentof_hincome_ownmortg: Homeowners with a mortgage showing total cost (including mortgage debt) as a percent of household income (%)
* hcost_as_perc_of_hincome_womortg: Homeowners without a mortgage showing total cost as a percent of household income.
*med_real_estate_taxes: The median real estate taxes paid by owners of homes in the area \\$
* family_members_per_hunit: The average size of related families members who live together in a housing unit. 
* median_num_ofrooms: The average number of rooms of total rooms for housing units in the area
* median_year_house_built: The average year the housing units were built in the area. This indicates the average age of housing units in the area.
* household_size_of_howners: For people who own their homes how many people on average are living in them whether they are part of family or related or not. 
* household_size_for_renters: The average size of a household for people who are renting.
* med_year_moved_in_for_owners: The median year that a home owner moved into their home
* med_year_renter_moved_in: The median year that a renter moved into their home 
* The following varialbes are monthly rental rates by size of  Rental in Bedrooms as a percentage
studio_1000_1499,studio_1500_more,
studio_750_999,
onebed_1000_1499,
onebed_1500_more,
onebed_750_999,
twobed_1000_1499,
twobed_1500_more,
twobed_750_999,
threebed_1000_1499,
threebed_1500_more,
threebed_750_999

In [31]:
results = []
not_found = []

In [16]:
def get_text_section(target_section,mysoup):
    """
    target_section:
    returns: 
    """
    found = mysoup.find_all('div',attrs = {'class':target_section})
    validsection = found[0]
    text_section = validsection.find_all(text=True)
    return text_section

In [17]:
def searchpattern(pattern,resultset):
    '''
    pattern:regex pattern
    resultset: A beautiful soup result set
    output housing variable of interest type string
    '''
    result = [i for i in resultset if re.search(pattern,i)]
    if result == []:
        result  = None
    elif result !=[]:
        result = (re.findall(pattern,result[0]))[0]
    return result

In [18]:
def refinestring(desiredreplacement,yourstring):
    """
    yourstring: a string which you extract only digits and decimals
    desiredreplacement: a dictonary definining your replacement values
    
    returns: your string with the desired replacements you request 
    """
    if yourstring  == None:
        return None
    elif yourstring != None:
        rep = dict((re.escape(k),v) for k,v in desiredreplacement.items())
        pattern =re.compile("|".join(rep.keys()))

        return float(pattern.sub(lambda m:rep[re.escape(m.group(0))],yourstring))

In [35]:
for aurl in city_urls:
    ua = UserAgent()
    t0 = time.time()
    response = requests.get(aurl,{"User-Agent": ua.random})
    
    if response.status_code != 200:
        not_found.append(aurl)
        continue
        
    response_delay = time.time() - t0
    time.sleep(10*response_delay)
    soup = BeautifulSoup(response.text,'html.parser')
    
    
    housing_text_section1 = get_text_section('section1',soup)
    housing_text_section2 = get_text_section('section2',soup)
    housing_text_section3 = get_text_section('section4',soup)
    
    #gathering variables
    if re.search('(.*)\sCalifornia',housing_text_section1[0]) == None:
        city_name = (pd.Series(aurl)).str.extract(r'Housing\/(.*?)-CA')
    elif re.search('(.*)\sCalifornia',housing_text_section1[0]) != None:
        city_name = re.search('(.*)\sCalifornia',housing_text_section1[0]).group(1)

    #for variables need to remove everything but numbers
    #figure1 
    housing_units = searchpattern('it\shas\s(\d.*?\d)\shousing\sunits\s',housing_text_section1)
    housing_units = refinestring({"$":"",",":"","%":""},housing_units)


    #figure2 
    housing_density = searchpattern('it\shas\s(\d.*?\d)\shousing\sdensity\s',housing_text_section1)
    housing_density = refinestring({"$":"",",":"","%":""},housing_density)

    #figure4 
    change_hunits = searchpattern('it\shas\s(.*?%)\schange\sin\s',housing_text_section1)
    change_hunits = refinestring({"$":"",",":"","%":""},change_hunits)

    #figure5
    percent_of_rent_to_total = searchpattern('it\shas\s(\d.*?%)\sRenter\sPercent\s',housing_text_section1)
    percent_of_rent_to_total = refinestring({"$":"",",":"","%":""},percent_of_rent_to_total)


    #figure 6
    owned_homes = searchpattern('it\shas\s(\d.*?%)\sOwner\sPercent\s',housing_text_section1)
    owned_homes = refinestring({"$":"",",":"","%":""},owned_homes)

    #figure 7 median home value (unique character  = $-numbers-commas)
    med_homeval = searchpattern('it\shas\s(.*?\d)\smedian\shome\svalue\s',housing_text_section2)
    med_homeval = refinestring({"$":"",",":"","%":""},med_homeval)

    #figure 8 median rental rate
    med_rental_rate = searchpattern('it\shas\s(.*?\d)\srental\srates\s',housing_text_section2)
    med_rental_rate = refinestring({"$":"",",":"","%":""},med_rental_rate)

    #figure 10 median owner costs
    med_owner_cost = searchpattern("it\shas\s(.*?\d)\smedian\sownership\scost\s",housing_text_section2)
    med_owner_cost = refinestring({"$":"",",":"","%":""},med_owner_cost)

    #figure 11
    med_own_cost_aspercentof_income = searchpattern('median\sowner\scost\sas\sa\spercent\sof\stotal\shousehold\sincome\sand\sit\shas\s(\d.*?%)',housing_text_section2)
    med_own_cost_aspercentof_income = refinestring({"$":"",",":"","%":""},med_own_cost_aspercentof_income)


    #figure 12 median home value as a percent of a median workers earnings
    med_hval_aspercentof_medearn = searchpattern("it\shas\s(\d.*?%)\smedian\shome\svalue\sas\s",housing_text_section2)
    med_hval_aspercentof_medearn = refinestring({"$":"",",":"","%":""},med_hval_aspercentof_medearn)


    #figure14
    med_hcost_ownmortg = searchpattern('depicts.*\$(.*?\d)\smedian',housing_text_section2)
    med_hcost_ownmortg = refinestring({"$":"",",":"","%":""},med_hcost_ownmortg)


    #figure 15
    med_hcost_own_wo_mortg = searchpattern('are.*\$(.*?\d)\smedian\shousing\s',housing_text_section2)
    med_hcost_own_wo_mortg = refinestring({"$":"",",":"","%":""},med_hcost_own_wo_mortg)

    #figure 17
    hcost_aspercentof_hincome_ownmortg= refinestring({"$":"",",":"","%":""},searchpattern('with\smortgages.*\s(\d.*?%)\scost',housing_text_section2))



    #figure18
    hcost_as_perc_of_hincome_womortg = refinestring({"$":"",",":"","%":""},searchpattern('This\sanalysis.*\s(\d.*?%)\scost',housing_text_section2))


    #figure 21 **
    med_real_estate_taxes = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\smedian\sreal\sestate\s',housing_text_section2))
    #figure 24 biggie see below

    #figure 28 format 3.0
    family_members_per_hunit = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\sfamily\smembers\sper\s',housing_text_section3))


    #figure 29 4.5
    median_num_ofrooms = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\saverage\snumber\sof\s',housing_text_section3))


    #figure 30
    median_year_house_built = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\syear\sbuilt\swhich\s',housing_text_section3))


    #figure 36
    household_size_of_howners = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\shomeowner\shousehold\ssize\s',housing_text_section3))


    #figure 37
    household_size_for_renters = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\srenter\shousehold\ssize\s',housing_text_section3))
    #figure 40
    med_year_moved_in_for_owners = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\shome',housing_text_section3))
    #figure41
    med_year_renter_moved_in = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\srenter',housing_text_section3))
    
    #scraping javascript table
    
    javascript = soup.find_all('script',attrs = {'type':'text/javascript'})
    need = javascript[4]
    needtext = str(need.find_all(text=True))
    if re.search('data24\s\=\sgoogle\.visualization\.arrayToDataTable\((.*?)\);',needtext) == None:
       
        studio_1000_1499 = None
        onebed_1000_1499 = None
        twobed_1000_1499 = None
        threebed_1000_1499 = None
        studio_1500_more = None
        onebed_1500_more = None
        twobed_1500_more = None
        threebed_1500_more = None
        studio_750_999 = None
        onebed_750_999 = None
        twobed_750_999 = None
        threebed_750_999 = None
    elif re.search('data24\s\=\sgoogle\.visualization\.arrayToDataTable\((.*?)\);',needtext) != None:
        
        fig24 = re.search('data24\s\=\sgoogle\.visualization\.arrayToDataTable\((.*?)\);',needtext).group(1)
        list_strs = re.findall(r'\[(.*?)\]',fig24)
        #gathering the variables in the table
        check1 = True
        check2 = True
        check3 = True
        for i,v in enumerate(list_strs):
            if '$1,000 to $1,499' in v:
                check1 = False
                val1 = re.findall('v\:(.*?\d),',v)
                studio_1000_1499 = float(val1[0])
                onebed_1000_1499 = float(val1[1])
                twobed_1000_1499 = float(val1[2])
                threebed_1000_1499 = float(val1[3])
            elif '$1,500 or more' in v:
                check2 = False
                val2 = re.findall('v\:(.*?\d),',v)
                studio_1500_more = float(val2[0])
                onebed_1500_more = float(val2[1])
                twobed_1500_more = float(val2[2])
                threebed_1500_more = float(val2[3])
            elif '$750 to $999' in v:
                check3 = False
                val3 = re.findall('v\:(.*?\d),',v)
                studio_750_999 = float(val3[0])
                onebed_750_999 = float(val3[1])
                twobed_750_999 = float(val3[2])
                threebed_750_999 = float(val3[3])

        if check1:
            studio_1000_1499 = None
            onebed_1000_1499 = None
            twobed_1000_1499 = None
            threebed_1000_1499 = None

        if check2:
            studio_1500_more = None
            onebed_1500_more = None
            twobed_1500_more = None
            threebed_1500_more = None

        if check3:
            studio_750_999 = None
            onebed_750_999 = None
            twobed_750_999 = None
            threebed_750_999 = None
    
    results.append((city_name,housing_units,housing_density,change_hunits,percent_of_rent_to_total,owned_homes,
           med_homeval,med_rental_rate,med_owner_cost,med_own_cost_aspercentof_income,med_hval_aspercentof_medearn,
          med_hcost_ownmortg,med_hcost_own_wo_mortg,hcost_aspercentof_hincome_ownmortg,hcost_as_perc_of_hincome_womortg,
          med_real_estate_taxes,family_members_per_hunit,median_num_ofrooms,median_year_house_built,household_size_of_howners,household_size_for_renters,
          med_year_moved_in_for_owners,med_year_renter_moved_in,studio_1000_1499,studio_1500_more,studio_750_999,onebed_1000_1499,onebed_1500_more,onebed_750_999,
          twobed_1000_1499,twobed_1500_more,twobed_750_999,threebed_1000_1499,threebed_1500_more,threebed_750_999))
    

In [38]:
housing = pd.DataFrame(results,columns =['city_name','housing_units','housing_density','change_hunits','percent_of_rent_to_total','owned_homes',
                'med_homeval','med_rental_rate','med_owner_cost','med_own_cost_aspercentof_income','med_hval_aspercentof_medearn',
                'med_hcost_ownmortg','med_hcost_own_wo_mortg','hcost_aspercentof_hincome_ownmortg','hcost_as_perc_of_hincome_womortg',
                'med_real_estate_taxes','family_members_per_hunit','median_num_ofrooms','median_year_house_built','household_size_of_howners','household_size_for_renters',
                'med_year_moved_in_for_owners','med_year_renter_moved_in','studio_1000_1499','studio_1500_more','studio_750_999','onebed_1000_1499','onebed_1500_more',
                 'onebed_750_999','twobed_1000_1499','twobed_1500_more','twobed_750_999','threebed_1000_1499','threebed_1500_more','threebed_750_999'] )

In [39]:
len(results),len(not_found)

(453, 5)

In [40]:
not_found

['https://www.towncharts.com/California/Housing/Carmel-By-the-Sea-city-CA-Housing-data.html',
 'https://www.towncharts.com/California/Housing/La-Cañada-Flintridge-city-CA-Housing-data.html',
 'https://www.towncharts.com/California/Housing/Paso-Robles-city-CA-Housing-data.html',
 'https://www.towncharts.com/California/Housing/St.-Helena-city-CA-Housing-data.html',
 'https://www.towncharts.com/California/Housing/Ventura-city-CA-Housing-data.html']

In [41]:
housing.to_csv(r'C:\\Users\\Crist\\AppData\\Local\\Temp\\American-Community-Survey-Project\\Towncharts_California_Housing.csv')

Merging the latitude and longitude from the kaggle dataset

In [61]:
os.getcwd()

'C:\\Users\\Crist\\Towncharts_Project\\California_Housing_Project'

In [62]:
cities_data = pd.read_csv("C:\\Users\\Crist\\Towncharts_Project\\California_Housing_Project\\datasets\\cal_cities_lat_long.csv")

In [63]:
housing = pd.read_csv("C:\\Users\\Crist\\Towncharts_Project\\California_Housing_Project//datasets/Towncharts_California_Housing.csv",index_col =0)

In [64]:
name_of_city =  []
for indx,city in enumerate(housing.city_name):
    ser_city = pd.Series(city)
    before_len = len(name_of_city)
    for aname in cities_data.Name:
        if ser_city[ser_city.str.contains(aname)].empty == False:
            name_of_city.append(aname)
            break
            
    after_len = len(name_of_city)
    if after_len == before_len:
        name_of_city.append(None)

In [66]:
housing['city'] = name_of_city 

In [68]:
california_housing = housing.merge(cities_data, left_on='city',right_on='Name')

In [69]:
california_housing = california_housing.drop(columns = ['city_name','Name'])

In [70]:
california_housing.to_csv('C:\\Users\\Crist\\Towncharts_Project\\California_Housing_Project\\datasets\\california_housing.csv')

In [ ]:
population_links = {}
for city in cities_data.Name:
    population_links[city] = 'http://www.towncharts.com/California/Demographics/{}-city-CA-Demographics-data.html'.format(city)
population_dict = {}
for key,value in population_links.items():
    ua = UserAgent()
    t0 = time.time()
    response = requests.get(value,{"User-Agent": ua.random})
    
    if response.status_code != 200:
        continue
    response_delay = time.time() - t0
    time.sleep(10*response_delay)
    soup = BeautifulSoup(response.text,'html.parser')
    population_text_section = get_text_section('section1',soup)
    population = searchpattern('it\shas\s(\d.*?\d)\spopulation\s',population_text_section)
    population = refinestring({"$":"",",":"","%":""},population)
    population_dict[key] = population
population_dat = pd.DataFrame(list(population_dict.items()),columns =['city','population'] )

In [100]:
population_dat = pd.DataFrame(list(population_dict.items()),columns =['city','population'] )

In [109]:
population_dat['city'] = population_dat['city'].str.replace('-',' ')

In [111]:
population_dat.head()

,city,population
0,Adelanto,32867.0
1,Agoura Hills,20736.0
2,Alameda,78246.0
3,Albany,19682.0
4,Alhambra,85168.0


In [112]:
#california_housing = housing.merge(cities_data, left_on='city',right_on='Name')
test_housing = california_housing.merge(population_dat, left_on='city',right_on='city')

In [115]:
test_housing.to_csv('C:\\Users\\Crist\\Towncharts_Project\\California_Housing_Project\\datasets\\california_housing.csv')

In [8]:
housing = pd.read_csv('C:\\Users\\Crist\\Towncharts_Project\\California_Housing_Project\\datasets\\california_housing.csv',index_col=0)

In [21]:
area_dat = (pd.read_csv('https://raw.githubusercontent.com/jakevdp/PythonDataScienceHandbook/master/notebooks/data/california_cities.csv',index_col=0)).loc[:,['city','area_total_km2']]

In [11]:
housing['city'] = housing['city'].str.replace(" ","")

In [26]:
test_dat = housing.merge(area_dat,left_on='city',right_on='city',how='left')

In [32]:
test_dat.to_csv('C:\\Users\\Crist\\Towncharts_Project\\California_Housing_Project\\datasets\\california_housing.csv')

In [ ]:
# thelist = (city_name,housing_units,housing_density,change_hunits,percent_of_rent_to_total,owned_homes,
#            med_homeval,med_rental_rate,med_owner_cost,med_own_cost_aspercentof_income,med_hval_aspercentof_medearn,
#           med_hcost_ownmortg,med_hcost_own_wo_mortg,hcost_aspercentof_hincome_ownmortg,hcost_as_perc_of_hincome_womortg,
#           med_real_estate_taxes,family_members_per_hunit,median_num_ofrooms,median_year_house_built,household_size_of_howners,household_size_for_renters,
#           med_year_moved_in_for_owners,med_year_renter_moved_in,studio_1000_1499,studio_1500_more,studio_750_999,onebed_1000_1499,onebed_1500_more,onebed_750_999,
#           twobed_1000_1499,twobed_1500_more,twobed_750_999,threebed_1000_1499,threebed_1500_more,threebed_750_999)

# thestringlist = ['city_name','housing_units','housing_density','change_hunits','percent_of_rent_to_total','owned_homes',
#                 'med_homeval','med_rental_rate','med_owner_cost','med_own_cost_aspercentof_income','med_hval_aspercentof_medearn',
#                 'med_hcost_ownmortg','med_hcost_own_wo_mortg','hcost_aspercentof_hincome_ownmortg','hcost_as_perc_of_hincome_womortg',
#                 'med_real_estate_taxes','family_members_per_hunit','median_num_ofrooms','median_year_house_built','household_size_of_howners','household_size_for_renters',
#                 'med_year_moved_in_for_owners','med_year_renter_moved_in','studio_1000_1499','studio_1500_more','studio_750_999','onebed_1000_1499','onebed_1500_more',
#                  'onebed_750_999','twobed_1000_1499','twobed_1500_more','twobed_750_999','threebed_1000_1499','threebed_1500_more','threebed_750_999']